In [1]:
import sys
import os

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Either hard code your path to the project, or set it as an environment variable
# build_project_path = os.environ['BUILD_PROJECT_PATH']

In [4]:
import os
os.environ['BUILD_PROJECT_PATH'] = r'C:\Users\L110006\OneDrive - Eli Lilly and Company\personal files\build project'
build_project_path = os.environ['BUILD_PROJECT_PATH']

In [5]:
synthetic_data_path = os.path.join(build_project_path, 'fine-tuning-build-project','synthetic_data')
print(synthetic_data_path)
data_path = os.path.join(synthetic_data_path, 'data')
print(data_path)

C:\Users\L110006\OneDrive - Eli Lilly and Company\personal files\build project\fine-tuning-build-project\synthetic_data
C:\Users\L110006\OneDrive - Eli Lilly and Company\personal files\build project\fine-tuning-build-project\synthetic_data\data


In [6]:
if synthetic_data_path not in sys.path:
    sys.path.append(synthetic_data_path)

In [7]:
pip install openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
from llm_requests import generate_prompt

seed_titles_df = pd.read_csv(os.path.join(data_path, 'seed_titles.csv'))

seed_titles = seed_titles_df['seed_title'].unique()

example_seed_titles = np.random.choice(seed_titles, 5)

example_prompt = generate_prompt(example_seed_titles)
print(example_prompt)

[{'role': 'system', 'content': 'You are an expert in recruitment, staffing and HR.'}, {'role': 'user', 'content': 'Your task is to generate \'in-the-wild\' job posting titles for given \'clean\' job titles. The \'in-the-wild\' titles should resemble real-world job postings as seen on sites like LinkedIn or company websites. These titles may include additional words, phrases, or abbreviations that are unnecessary or irrelevant to classifying the job into a standard taxonomy.\n\n**Instructions:**\nor this list of \'clean\' job titles, generate a list of 2 \'in-the-wild\' job titles for each.\n\nIt is important to:\n\n- Format your responses consistently (within and between queries)\n- Vary the sentence structure/order, length, and additional information in the \'in-the-wild\' titles.\n- Rarely include a company name, and if you do, choose an appropriate real company.\n- You may include locations, departments, or sectors (e.g., "East Coast Operations", "Global Data Strategy") infrequently

C:\Users\L110006\OneDrive - Eli Lilly and Company\personal files\build project\fine-tuning-build-project\synthetic_data\llm_requests.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


<h3>Experiment with different models and see how they compare!</h3>

In [9]:
model_name = 'Meta-Llama-3.3-70B-Instruct'
# model_name = 'Meta-Llama-3.1-405B-Instruct'

In [10]:
#import os
#
##os.environ["SAMBANOVA_API_KEY"] = "8841f010-0daa-4701-bdaf-89937409bac7"
#os.environ["SAMBANOVA_API_KEY"] = "829993a9-ff57-414c-912d-58212968888b"

In [11]:
from llm_requests import get_client

client = get_client()

In [12]:
from llm_requests import async_make_api_call

example_response = await async_make_api_call(client, model_name, example_prompt, perturbation_std=0.1)

In [13]:
from gpt_parsing import parse_gpt_response

parsed_output = parse_gpt_response(example_response.choices[0].message.content, 5, 5)

In [14]:
for seed_title, response in zip(example_seed_titles, parsed_output):
    print(f'Variations of: {seed_title}:')
    print('-------------------')
    for i, variation in enumerate(response):
        print(f'{i+1}: {variation}')

    print('\n\n')

Variations of: Layout Technician (Layout Tech):
-------------------
1: Layout Technician (CAD Design) - Architecture Firm
2: Entry-Level Layout Tech - Furniture Manufacturing
3: Senior Layout Technician - Industrial Automation
4: Layout Tech - Mechanical Engineering (Temporary Position)
5: Technical Layout Specialist - Exhibits and Displays



Variations of: Broadcast Meteorologist:
-------------------
1: Morning Broadcast Meteorologist - ABC Affiliate
2: Chief Meteorologist - NBC News Team (Weekend Shifts)
3: Broadcast Meteorologist (Part-Time) - Severe Weather Coverage
4: On-Air Meteorologist - Live Forecasting Experience Required
5: TV Broadcast Meteorologist - Storm Chaser Opportunity



Variations of: Early Childhood Teacher:
-------------------
1: Lead Early Childhood Teacher - Pre-K Program (Head Start)
2: Infant/Toddler Early Childhood Teacher - Full-Time, Benefits
3: Early Childhood Special Education Teacher - Autism Focus
4: Preschool Early Childhood Teacher - Music and Arts 

In [16]:
#from llm_requests import async_main_stubborn
#
#output_dict_path = os.path.join(data_path, 'jitter_responses.pkl')
#response_dict = await async_main_stubborn(
#    all_query_titles=seed_titles,
#    client=client,
#    model_name=model_name,
#    output_path=output_dict_path,
#    chunk_size=10,
#    num_examples_per_title=2,
#    delay=2,
#    giveup_after=5,
#    
#)

  0%|          | 0/2922 [00:00<?, ?it/s]

Attempt 1 for chunk 394


RateLimitError: Error code: 429 - {'error': {'code': None, 'message': 'Rate limit exceeded', 'param': None, 'type': 'rate_limit_exceeded'}}

In [17]:
import os
import pandas as pd
import numpy as np
from llm_requests import generate_prompt

# Parameters
seed_file = os.path.join(data_path, "seed_titles.csv")
num_examples = 5       # number of seed titles per prompt
num_prompts = 3        # number of prompts you want to generate
random_seed = 42       # set to None if you want different results each run

# Optional reproducibility
if random_seed is not None:
    np.random.seed(random_seed)

# Load seed titles
if not os.path.exists(seed_file):
    raise FileNotFoundError(f"Seed file not found: {seed_file}")

seed_titles_df = pd.read_csv(seed_file)

if "seed_title" not in seed_titles_df.columns:
    raise KeyError("CSV file must have a column named 'seed_title'")

seed_titles = seed_titles_df["seed_title"].dropna().unique()

# Generate multiple prompts
all_prompts = []
for _ in range(num_prompts):
    example_seed_titles = np.random.choice(seed_titles, size=num_examples, replace=False)
    prompt = generate_prompt(example_seed_titles)
    all_prompts.append(prompt)

# Print results
for i, p in enumerate(all_prompts, start=1):
    print(f"\n--- Prompt {i} ---")
    print(p)



--- Prompt 1 ---
[{'role': 'system', 'content': 'You are an expert in recruitment, staffing and HR.'}, {'role': 'user', 'content': 'Your task is to generate \'in-the-wild\' job posting titles for given \'clean\' job titles. The \'in-the-wild\' titles should resemble real-world job postings as seen on sites like LinkedIn or company websites. These titles may include additional words, phrases, or abbreviations that are unnecessary or irrelevant to classifying the job into a standard taxonomy.\n\n**Instructions:**\nor this list of \'clean\' job titles, generate a list of 2 \'in-the-wild\' job titles for each.\n\nIt is important to:\n\n- Format your responses consistently (within and between queries)\n- Vary the sentence structure/order, length, and additional information in the \'in-the-wild\' titles.\n- Rarely include a company name, and if you do, choose an appropriate real company.\n- You may include locations, departments, or sectors (e.g., "East Coast Operations", "Global Data Strat

In [18]:
import pickle

output_dict_path = os.path.join(data_path, 'jitter_responses.pkl')

with open(output_dict_path, 'rb') as f:
    response_dict = pickle.load(f)

In [19]:
jitter_df = {
    'jittered_title': [],
    'seed_title': [],
}

for seed_title, jittered_titles in response_dict.items():
    for jittered_title in jittered_titles:
        jitter_df['jittered_title'].append(jittered_title)
        jitter_df['seed_title'].append(seed_title)

jitter_df = pd.DataFrame(jitter_df)

jitter_df = jitter_df.merge(seed_titles_df, on='seed_title', how='left')
jitter_df.sample(5)

,jittered_title,seed_title,onet_code,onet_name
1018,Adapted Physical Education Specialist (APE) - ...,Adapted Physical Education Specialist (APE Spe...,25-2059.01,Adapted Physical Education Specialists
283,"EVP, Chief Financial Officer - Lead Our Financ...",Executive Vice President (EVP),11-1011.00,Chief Executives
3265,Senior Bike Mechanic Wanted - High-Volume Repa...,Bike Mechanic,49-3091.00,Bicycle Repairers
5822,Signal Maintainer - Rail Transportation (Multi...,Signal Maintainer,49-9097.00,Signal and Track Switch Repairers
4312,Senior Psychometric Consultant - Assessment So...,Psychometric Consultant,15-2041.00,Statisticians


In [20]:
jitter_df.to_csv(os.path.join(data_path, 'jittered_titles.csv'), index=False)